# Inference Sample for EquiDock

SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: LicenseRef-NvidiaProprietary

NVIDIA CORPORATION, its affiliates and licensors retain all intellectual
property and proprietary rights in and to this material, related
documentation and any modifications thereto. Any use, reproduction,
disclosure or distribution of this material and related documentation
without an express license agreement from NVIDIA CORPORATION or
its affiliates is strictly prohibited.



### Prerequisite

- Linux OS
- Pascal, Volta, Turing, or an NVIDIA Ampere architecture-based GPU.
- NVIDIA Driver
- Docker

#### Import

Components for inferencing are part of the BioNeMo EquiDock source code. This notebook demonstrates the use of these components.

In [ ]:
import os
from pytriton.client import ModelClient
from zipfile import ZipFile
import tempfile

In [ ]:
from typing import List
from pathlib import Path
import os

try:
    BIONEMO_HOME: Path = Path(os.environ['BIONEMO_HOME']).absolute()
except KeyError:
    print("Must have BIONEMO_HOME set in the environment! See docs for instructions.")
    raise

config_path = BIONEMO_HOME / "examples" / "protein" / "equidock" / "conf"
print(f"Using model configuration at: {config_path}")
assert config_path.is_dir()

## Inputs

### Set input ligand and receptor PDB files

In [ ]:
## In this notebook example, we will use ligand and receptors files in the test directory
test_folder_location = os.path.join(os.path.dirname(os.getcwd()),  "../../../") # test directory
zip_file_path = "tests/equidock_test_data/test_sets_pdb/dips_test_random_transformed/random_transformed" 

In [ ]:
ligands_zip = os.path.join(test_folder_location, zip_file_path, "ligands.zip")
receptors_zip = os.path.join(test_folder_location, zip_file_path, "receptors.zip")

# Temporary output folder
tmp_output_folder = tempfile.TemporaryDirectory()

# Select ligand filename for extraction
ligand_file = "a9_1a95.pdb1_3.dill_l_b.pdb"
with ZipFile(ligands_zip, "r") as zipper:
    zipper.extract(ligand_file, tmp_output_folder.name)

print(f"Ligand extracted to {os.path.join(tmp_output_folder.name, ligand_file)}")
# Receptor filename for extraction
receptor_file = "a9_1a95.pdb1_3.dill_r_b.pdb"
with ZipFile(receptors_zip, "r") as zipper:
    zipper.extract(receptor_file, tmp_output_folder.name)
print(f"Receptor extracted to {os.path.join(tmp_output_folder.name, receptor_file)}")

## Ouputs
### Set output complex PDB file

In [ ]:
out_file = "a9_1a95.pdb1_3.equidock_complex.pdb"
print(f"Output will be written to {os.path.join(tmp_output_folder.name, out_file)}")

### Format input/outputs for `pytriton` client

In [ ]:
ligand_filename = os.path.join(tmp_output_folder.name, ligand_file)

receptor_filename = os.path.join(tmp_output_folder.name, receptor_file)

out_filename = os.path.join(tmp_output_folder.name, out_file)

###  Use ModelClient to run inference on the server

EquiDock predicted complex written to `out_filename`

In [ ]:
from bionemo.triton.utils import load_model_config

cfg = load_model_config(config_path, config_name="infer.yaml")

In [ ]:
from bionemo.triton.utils import load_model_for_inference
from bionemo.model.protein.equidock.infer import EquiDockInference

inferer = load_model_for_inference(cfg, interactive=True)

print(f"Loaded a {type(inferer)}")
assert isinstance(inferer, EquiDockInference)

In [ ]:
predicted_complex = inferer.infer(ligand_filename, receptor_filename, cfg.data)

In [ ]:
predicted_complex.to_pdb(path=out_filename, records=["ATOM"], gz=False)

In [ ]:
# Install py3Dmol for visualization
try:
    import py3Dmol
except:
    !pip install py3Dmol
    import py3Dmol

In [ ]:
# Load ligand, receptor, and predicted complex files for visualization
with open(os.path.join(tmp_output_folder.name, receptor_file)) as ifile:
    receptor = "".join([x for x in ifile])
    
with open(os.path.join(tmp_output_folder.name, ligand_file)) as ifile:
    ligand = "".join([x for x in ifile])

with open(os.path.join(tmp_output_folder.name, out_file)) as ifile:
    equidock_complex = "".join([x for x in ifile])

In [ ]:
print("Receptor")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(receptor)
view.setStyle({'model': -1}, {"cartoon": {'color': 'purple'}})
view.zoomTo()
view.show()

In [ ]:
print("Ligand")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(ligand)
view.setStyle({'model': -1}, {"cartoon": {'color': 'green'}})
view.zoomTo()
view.show()

In [ ]:
print("EquiDock complex")
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(receptor)
view.setStyle({'model': 0}, {"cartoon": {'color': 'purple'}})
view.addModelsAsFrames(equidock_complex)
view.setStyle({'model': 1}, {"cartoon": {'color': 'green'}})
view.zoomTo()
view.show()